In [1]:
import pandas as pd

peres = pd.read_csv('./dataset_brut/sts_peres.csv', sep=";")
bonparte_st_germain = pd.read_csv('./dataset_brut/bonparte_st_germain.csv', sep=";")

In [2]:
(peres.isnull().sum()/peres.shape[0])*100

Identifiant arc               0.000000
Libelle                       0.000000
Date et heure de comptage     0.000000
Débit horaire                79.289877
Taux d'occupation            79.289877
Etat trafic                   0.000000
Identifiant noeud amont       0.000000
Libelle noeud amont           0.000000
Identifiant noeud aval        0.000000
Libelle noeud aval            0.000000
Etat arc                      0.000000
Date debut dispo data         0.000000
Date fin dispo data           0.000000
geo_point_2d                  0.000000
geo_shape                     0.000000
dtype: float64

In [3]:
(bonparte_st_germain.isnull().sum()/bonparte_st_germain.shape[0])*100

Identifiant arc              0.000000
Libelle                      0.000000
Date et heure de comptage    0.000000
Débit horaire                1.910209
Taux d'occupation            2.762789
Etat trafic                  0.000000
Identifiant noeud amont      0.000000
Libelle noeud amont          0.000000
Identifiant noeud aval       0.000000
Libelle noeud aval           0.000000
Etat arc                     0.000000
Date debut dispo data        0.000000
Date fin dispo data          0.000000
geo_point_2d                 0.000000
geo_shape                    0.000000
dtype: float64

In [4]:
bonparte_st_germain['Date et heure de comptage'] = pd.to_datetime(
    bonparte_st_germain['Date et heure de comptage'],
    errors='coerce',
    utc=True
).dt.tz_convert('Europe/Paris').dt.tz_localize(None)
print(len(bonparte_st_germain['Date et heure de comptage'].unique()))

9266


In [5]:
peres['Date et heure de comptage'] = pd.to_datetime(
    peres['Date et heure de comptage'],
    errors='coerce',
    utc=True
).dt.tz_convert('Europe/Paris').dt.tz_localize(None)
print(len(peres['Date et heure de comptage'].unique()))

9266


In [6]:
bonparte_st_germain = bonparte_st_germain.sort_values('Date et heure de comptage')
peres = peres.sort_values('Date et heure de comptage')

df_merge = pd.merge(bonparte_st_germain,peres, on='Date et heure de comptage', how = "inner")

df_merge.to_csv("./mrged.csv", sep=";")
print(df_merge.columns.to_list())

['Identifiant arc_x', 'Libelle_x', 'Date et heure de comptage', 'Débit horaire_x', "Taux d'occupation_x", 'Etat trafic_x', 'Identifiant noeud amont_x', 'Libelle noeud amont_x', 'Identifiant noeud aval_x', 'Libelle noeud aval_x', 'Etat arc_x', 'Date debut dispo data_x', 'Date fin dispo data_x', 'geo_point_2d_x', 'geo_shape_x', 'Identifiant arc_y', 'Libelle_y', 'Débit horaire_y', "Taux d'occupation_y", 'Etat trafic_y', 'Identifiant noeud amont_y', 'Libelle noeud amont_y', 'Identifiant noeud aval_y', 'Libelle noeud aval_y', 'Etat arc_y', 'Date debut dispo data_y', 'Date fin dispo data_y', 'geo_point_2d_y', 'geo_shape_y']


In [7]:
import pandas as pd
import numpy as np

# Tri (ok)
# bac = bac.sort_values('Date et heure de comptage')
# peres = peres.sort_values('Date et heure de comptage')

# Jointure stricte sur l'horodatage
df = df_merge

# Nettoyage des deux colonnes (virgules décimales, % éventuel, espaces)
for col in ["Taux d'occupation_x", "Taux d'occupation_y"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.replace(',', '.', regex=False)
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Ne garder que les lignes où les deux valeurs existent
df_valid = df.dropna(subset=["Taux d'occupation_x", "Taux d'occupation_y"])
print(df_valid.shape[0])

# Contrôles de validité
n = len(df_valid)
if n < 2:
    raise ValueError("Pas assez de paires valides après nettoyage (n < 2).")

if df_valid["Taux d'occupation_x"].nunique() == 1 or df_valid["Taux d'occupation_y"].nunique() == 1:
    raise ValueError("L'une des séries est constante : la corrélation de Pearson est indéfinie.")

# Corrélation de Pearson
r = df_valid["Taux d'occupation_x"].corr(df_valid["Taux d'occupation_y"])
print("Corrélation :", r)

1898
Corrélation : 0.5993512654172726


In [8]:
df_valid_debit = df.dropna(subset=["Débit horaire_x", "Débit horaire_y"])
print(df_valid_debit.shape[0])
r = df_valid_debit["Débit horaire_x"].corr(df_valid_debit["Débit horaire_y"])
print(r)

1885
0.8932025415739622
